# Car Decor Sales Forecasting - ChromeAccessories

###### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
from math import sqrt

# Connecting Python to MySQL for fetching data 
import mysql.connector

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

###### MySQL Connection to fetch data

In [ ]:
try:
    connection = mysql.connector.connect(host='localhost',
                                        database='car_decors',
                                         user='root',
                                         password='***********')

    sql_select_Query = "SELECT * FROM decorsales"
    cursor = connection.cursor()
    cursor.execute(sql_select_Query)
    columns = len(cursor.description)
    columns = [i[0] for i in cursor.description]
    print(columns)

    # get all records
    records = cursor.fetchall()
    print("Total number of rows in table: ", cursor.rowcount)

except mysql.connector.Error as e:
    print("Error reading data from MySQL table", e)

### Data Cleaning and Exploratory Data Analysis

###### Converting fetched records to Pandas dataframe

In [ ]:
records = np.array(records)
records = records[:,0:25]
decor_sales=pd.DataFrame(records,columns=columns)

###### Type Casting Date and other features

In [ ]:
decor_sales.dtypes
decor_sales.Date = pd.to_datetime(decor_sales.Date)
decor_sales.iloc[:,1:] = decor_sales.iloc[:,1:].astype("int32")
decor_sales.dtypes

###### Creating Subset of Decor Sales Dataset and resampling Monthly Time Series

In [ ]:
df = decor_sales
df = df.set_index('Date')
df = df.resample("MS").sum()

###### Data Visualization

In [ ]:
plt.rc("figure", figsize=(16,8))
sns.set_style('darkgrid')

###### Rolling statistics to observe variation in mean and standard deviation.

In [ ]:
timeseries = df ['ChromeAccessories']
timeseries.rolling(12).mean().plot(label='12 Month Rolling Mean', marker='.')
timeseries.rolling(12).std().plot(label='12 Month Rolling Std', marker='.')
timeseries.plot(marker='.')
plt.title('Rolling Statistics to observe variation in Mean and Standard Deviation', fontsize = 18, fontweight = 'bold')
plt.xlabel('Year', fontsize = 14)
plt.ylabel('Sales (Number of Units)', fontsize = 14)
plt.legend()

###### Checking Seasonalty and Trend components for the feature

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
add = seasonal_decompose(df["ChromeAccessories"],model="additive",period=12)
add.plot();

##### Checking for Data Stationarity using Augmented Dickey-Fuller Test

In [ ]:
from statsmodels.tsa.stattools import adfuller

def check_adf(time_series):
    test_result = adfuller(df['ChromeAccessories'])
    print ('ADF Test:')
    labels = ['ADF Statistic','p-value','No. of Lags Used','Number of Observations Used']

    for value,label in zip(test_result,labels):
        print (label+': '+str(value)+str("\n"))
        if test_result [1] <= 0.05:
            print ("Reject null hypothesis; Data is stationary")
        else:
            print ("Fail to reject H0; Data is non-stationary")

In [ ]:
check_adf(df['ChromeAccessories'])

# Adfuller test Results for all variables

In [ ]:
from statsmodels.tsa.stattools import adfuller
def adfuller_parameter(x):
    P = []
    columns = []
    used_lag = []
    for i in x.columns:
        test_stats,p,used_lags,nobs,critical_value,ic_best = adfuller(x[i])
        columns.append(i)
        P.append(p)
        used_lag.append(used_lags)
    return pd.DataFrame({"COLUMNS":columns,"P_VALUE":P,"MAX_USED_LAG":used_lag})

adfuller_parameter(df)

##### Hyper-parameter Tuning # Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) plots

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm

fig, ax = plt.subplots(1,2, figsize=(15,5))
sm.graphics.tsa.plot_acf(df["ChromeAccessories"], lags=12, title = 'ACF Plot', ax=ax[0])
sm.graphics.tsa.plot_pacf(df["ChromeAccessories"], lags=12, title = 'PACF Plot',ax=ax[1])
plt.show()

### Model Building - SARIMA Model ( Seasonal ARIMA Model )

###### Train Test Split

In [ ]:
train_df = df["ChromeAccessories"].iloc[0:int(len(df)*.95)] #train model with approx 95% data
test_df = df["ChromeAccessories"].iloc[int(len(train_df)):] #test model with 5% data

print("Train_df : ",len(train_df))
print("Test_df : ",len(test_df))

###### User Defined Function to calculate the MAPE value 

In [ ]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

###### Automated Hyperparameter tuning

In [ ]:
 import itertools as i 
p = range(0,3) 
d = range(0,2)
q = range(0,3)

pdq_combo = list(i.product(p,d,q)) #this will all combination of p,d,q throgh a tuple 

error = []
aic_sarima = []
order_arima = []
order_sarima = []
seasonality = 12
for pdq in pdq_combo:
    for PDQ in pdq_combo:
        try:
            SEASONAL_ORDER = list(PDQ)
            SEASONAL_ORDER.append(seasonality)
            model = sm.tsa.SARIMAX(train_df,order=(pdq),seasonal_order=tuple(SEASONAL_ORDER))
            result = model.fit(disp=0)
            pred = result.predict(start=len(train_df),end=len(df)-1)
            eror = mape(test_df,pred)
            aic_sarima.append(result.aic)
            order_arima.append(pdq)
            order_sarima.append(tuple(SEASONAL_ORDER))
            error.append(eror)
        except:
            continue

In [ ]:
# Creating a dataframe of seasonality orders and errors 
df_error = pd.DataFrame({"arima_order":order_arima,"sarima_order": order_sarima,"error":error,"aic":aic_sarima})
df_error = df_error.sort_values(by="error",ascending = True)
df_error.reset_index(inplace=True,drop=True)

In [ ]:
## best parameter selection
p_d_q = df_error.iloc[0,0] #choosing best parameter for arima order
P_D_Q = df_error.iloc[0,1] #choosing best parameter for seasonal  order

In [ ]:
## best parameter selection
print("Best p_d_q parameter : ", p_d_q)
print("Best P_D_Q parameter : ", P_D_Q)

###### Model with best parameter

In [ ]:
sarima_model = sm.tsa.SARIMAX(train_df, order=(p_d_q), seasonal_order=(P_D_Q))
sarima_results = sarima_model.fit(disp = 0)
sarima_pred = sarima_results.predict(start=test_df.index[0],end=test_df.index[-1])
sarima_pred_large = sarima_results.predict(start=75,end=86,dynamic=True)

In [ ]:
print(sarima_results.summary())
sarima_diagnostics = sarima_results.plot_diagnostics(figsize=(16,8))

In [ ]:
# Predicted values
# Point estimation
sarima_prediction = sarima_results.get_prediction(start = test_df.index[0], end = test_df.index[-1], dynamic = True, full_results = True)
sarima_point_estimation = sarima_prediction.predicted_mean
sarima_point_estimation

In [ ]:
#Checking MAPE
mape(test_df, sarima_point_estimation)

In [ ]:
# At 95% confidence interval
sarima_pred_range = sarima_prediction.conf_int(alpha = 0.05)
sarima_pred_range

In [ ]:
# Ploting Sarima Prediction
plt.plot(train_df,color="g",label="Train Data", marker='.')
plt.plot(test_df,color="b",label="Test Data", marker='.')
plt.plot(sarima_point_estimation,color="r",label="Forecast (Test Data)", marker='.')
plt.figtext(0.13, 0.15, '\nMAPE     :  {} \nSARIMA :  {},{} \nAIC        :  {}'.format(mape(test_df, sarima_point_estimation), p_d_q, P_D_Q, sarima_results.aic, fontsize = 11))
plt.fill_between(sarima_pred_range.index,sarima_pred_range.iloc[:,0],sarima_pred_range.iloc[:,1],color='b',alpha=.2)
plt.legend(loc="upper right")

### Holt Winters Exponential Smoothing with Additive Seasonality and Additive Trend

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing # 

hwe_model_add_add = ExponentialSmoothing(train_df, seasonal ="add", trend = "add", seasonal_periods = 12).fit()
pred_hwe_add_add = hwe_model_add_add.predict(start = test_df.index[0], end = test_df.index[-1])

In [ ]:
pred_hwe_add_add

###### Plotting Holt Winters Model 

In [ ]:
plt.plot(train_df,color="g",label="Train Data")
plt.plot(test_df,color="b",label="Test Data")
plt.plot(pred_hwe_add_add,color="r",label="Forecast (Test Data)")
plt.suptitle('Model : Holt Winters', fontsize = 12, fontweight = 'bold')
plt.title('Car Decors - ANDROID HEAD UNITS', fontsize = 18, fontweight = 'bold')
plt.figtext(0.13, 0.14, '\nMAPE     :  {} \nAIC        :  {}'.format(mape(test_df, pred_hwe_add_add), hwe_model_add_add.aic))
plt.xlabel('Year', fontsize = 14)
plt.ylabel('Sales (Number of Units)', fontsize = 14)
plt.legend(loc="best")

In [ ]:
mape(test_df, pred_hwe_add_add) 

### FB Prophet Model

In [ ]:
# Loading Libraries
from fbprophet import Prophet
from fbprophet.plot import plot_plotly

df1 = decor_sales
df1 = df1.set_index('Date')
df1 = df1.resample("MS").sum()
df1.reset_index(inplace=True)

In [ ]:
train_df1 = df1[["Date","ChromeAccessories"]].iloc[0:int(len(df1)*.95)] #train model with approx 95% data
test_df1 = df1[["Date","ChromeAccessories"]].iloc[int(len(train_df1)):] #test model with 5% data

print("Train : ",len(train_df1))
print("Test : ",len(test_df1))

In [ ]:
train_df1.columns = ["ds","y"]
test_df1.columns = ["ds","y"]

In [ ]:
# Fitting the Model
prophet_model = Prophet().fit(train_df1)

In [ ]:
# Define the period for which we want a prediction
future = list()
for i in range(1, 5):
	date = '2021-%02d' % i
	future.append([date])
future = pd.DataFrame(future)
future.columns = ['ds']
future['ds']= pd.to_datetime(future['ds'])
future 

In [ ]:
forecast = prophet_model.predict(future)
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

In [ ]:
test_df1=test_df1.set_index("ds")
train_df1 = train_df1.set_index("ds")
forecast=forecast.set_index("ds")

In [ ]:
plt.style.use("ggplot")
plt.plot(train_df1['y'],color="r",label="Train Data")
plt.plot(test_df1['y'],color="b",label="Test Data")
plt.plot(forecast["yhat"],color="g",label="Forecast (Test Data)")
plt.grid( linestyle='-', linewidth=2)
plt.legend(loc="best")

In [ ]:
# MAPE
mape(test_df1['y'], forecast['yhat'])

In [ ]:
#RMSE
sqrt(mean_squared_error(test_df1['y'], forecast['yhat'].tail(4)))

### Auto Time Series Model

In [ ]:
from auto_ts import auto_timeseries

In [ ]:
train_df2 = train_df1
test_df2 = test_df1

In [ ]:
ts_model = auto_timeseries( score_type='rmse', time_interval='MS', non_seasonal_pdq=(12,12,12), seasonality=True, seasonal_period=12, model_type="best", verbose=2)

In [ ]:
ts_model.fit(traindata= train_df2, ts_column="ds", target="y")

In [ ]:
ts_model.get_leaderboard()

In [ ]:
ts_model.plot_cv_scores()

In [ ]:
future_predictions = ts_model.predict(test_df2, model='best')
future_predictions

In [ ]:
# define the period for which we want a prediction
ts_future = list()
for i in range(1, 5):
	date = '2021-%02d' % i
	ts_future.append([date])
ts_future = pd.DataFrame(ts_future)
ts_future.columns = ['ds']
ts_future['ds']= pd.to_datetime(ts_future['ds'])

In [ ]:
ts_model.predict(ts_future)

In [ ]:
mape(test_df2["y"],future_predictions["yhat"])

### Models Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error as mse
print("\nSARIMA Trend          :  ", p_d_q)
print("SARIMA Seasonal Order :  ", P_D_Q)
print("SARIMA AIC            :  ", sarima_results.aic)
print("SARIMA RMSE           :  ", np.sqrt(mse(test_df,sarima_point_estimation)))
print("SARIMA MAPE           :  ", mape(test_df, sarima_point_estimation))
print("\nHolt Winters AIC      :  ", hwe_model_add_add.aic)
print("Holt Winters RMSE     :  ", np.sqrt(mse(test_df,pred_hwe_add_add)))
print("Holt Winters MAPE     :  ", mape(test_df, pred_hwe_add_add))
print("\nFB Prophet RMSE       :  ", sqrt(mean_squared_error(test_df1['y'], forecast['yhat'])))
print("FB Prophet MAPE       :  ", mape(test_df1['y'], forecast['yhat']))
print("\nAuto Time Series: \n  ", ts_model.get_leaderboard())
print("Auto Time Series MAPE       :  ", mape(test_df2["y"],future_predictions["yhat"]))

In [ ]:
sarima = mape(test_df, sarima_point_estimation)
hwinters = mape(test_df, pred_hwe_add_add)
fbprophet = mape(test_df1['y'], forecast['yhat'])
autots = mape(test_df2["y"],future_predictions["yhat"])

mape_data = {'models':['SARIMA','HOLTWINTERS','FB_PROPHET','AUTO_TS'], 'name':['sarima_model', 'hwe_model_add_add','prophet_model','ts_model'],'mape':[sarima, hwinters, fbprophet, autots]}
mape_error = pd.DataFrame(mape_data)
mape_error = mape_error.sort_values(by="mape",ascending = True)
mape_error.reset_index(inplace=True,drop=True)
#best_model = mape_error.iloc[0,0]
print('\033[1m'+"Best Model with lowest MAPE : ", mape_error.iloc[0,0] + " ( " + mape_error.iloc[0,1] + " ) " + '\033[0m')
print("\nMAPE ERRORS :\n\n", mape_error)

##### Saving Model

In [ ]:
import pickle
filename = 'sarima_ca.pkl'
pickle.dump(sarima_model, open(filename, 'wb'))

######  Testing saved Model for prediction

In [ ]:
####### Model summary and diagnstics plot #######
with open(filename, "rb") as file:
    load_model = pickle.load(file)
    
result = load_model.fit()
#print(result.summary())
#diagnostics = result.plot_diagnostics(figsize=(16,8))

In [ ]:
pred = result.get_prediction(start = 76, end = 87, dynamic = False)

# Point estimation
prediction = pred.predicted_mean
prediction = round(prediction)
prediction

In [ ]:
# Ploting final Sarima Prediction
plt.plot(df['ChromeAccessories'],color="g",label="Actual", marker='.')
plt.plot(prediction,color="r",label="Forecast", marker='.')
plt.suptitle('Model : SARIMA', fontsize = 12, fontweight = 'bold')
plt.title('Car Decors - Chrome Accessories', fontsize = 18, fontweight = 'bold')
plt.figtext(0.13, 0.14, '\nMAPE     :  {} \nAIC        :  {}'.format(mape(test_df, sarima_point_estimation), sarima_results.aic))
plt.xlabel('Year', fontsize = 14)
plt.ylabel('Sales (Number of Units)', fontsize = 14)
plt.legend(loc="best")

### Closing connection to MySQL and clearing variables from memory.

In [ ]:
#if connection.is_connected():
#    connection.close()
#    cursor.close()
#    print("MySQL connection is closed")

# Clear all variables from memory
#globals().clear()

#####################################################################